## Load packages

In [1]:
import sys
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math as mt
from datetime import datetime, timedelta
from math import sin, cos, pi  # rotating regions
from math import floor  # truncating naics codes
import numba  # speed up data transform with JIT compilation

## Define Constants

In [11]:
# Root directory for dataset
dataroot = # where you save and load data

# "grid coordinates" created in R are in meters
cell_width = 1 * 0.025 * 1.609344 * 1000  # cell width in meters (convert from miles)
size_potential = 10  # potential locations: num_width_potential x num_width_potential
size_padding = 20  # number of padding cells on each side of potential grid
nc = 25 #3  # number of channels

BATCH_SIZE_real = 96  # regions with missing grocery store per batch #96
BATCH_SIZE_fill = 16 #16  # regions with real location filled in (-> no missing) per batch
BATCH_SIZE_random = 8  # random regions (-> no missing) per batch
BATCH_SIZE = BATCH_SIZE_real + BATCH_SIZE_fill + BATCH_SIZE_random

frac_train_real = 1.0  # fraction of real regions to use for training
frac_train_random = 1.0  # fraction of random (unrealized) regions to use for training

use_saved_model = False #True
saved_model_filename = # filename if choose to use saved model

use_cuda = True

EPOCHS = 10
sampleTotNum = 418
ITERS = 100 

labelThresh = 1000 # start from where there are counterfactual openings

weight_fine = 1
weight_plain = 1

eval_weight_fine = 1
eval_weight_plain = 1

fineSampleNumber = 418

In [12]:
print('BATCH_SIZE: ' + str(BATCH_SIZE))
print('cell_width: ' + str(round(cell_width)) + 'm')
# print('prob_none: ' + str(round(100 * prob_none)) + '%')

BATCH_SIZE: 120
cell_width: 40m


## Read in data

In [13]:
dict_S_I = dict()
dict_S_I_restaurant = dict()

# read in data of businesses near each grocery store
with open(dataroot+'grid_S_I.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        i_id = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])

        if slist[4] == 'NA':
            popu = -1
        else:
            popu = float(slist[4])

        if slist[5] == 'NA':
            sau16 = -1
        else:
            sau16 = float(slist[5])

        if slist[6] == 'NA':
            sa1664 = -1
        else:
            sa1664 = float(slist[6])
                    
        if slist[7] == 'NA':
            sw = -1
        else:
            sw = float(slist[7])
                    
        if slist[8] == 'NA':
            sc = -1
        else:
            sc = float(slist[8])
                    
        if slist[9] == 'NA':
            scp = -1
        else:
            scp = float(slist[9])
                    
        if slist[10] == 'NA':
            mhh = -1
        else:
            mhh = float(slist[10])
                    
        if slist[11] == 'NA':
            suem = -1
        else:
            suem = float(slist[11])
                    
        if slist[12] == 'NA':
            mhhv = -1
        else:
            mhhv = float(slist[12])
                    
        if slist[13] == 'NA':
            stc = -1
        else:
            stc = float(slist[13])
                    
        if slist[14] == 'NA':
            stp = -1
        else:
            stp = float(slist[14])
                    
        if slist[15] == 'NA':
            sth = -1
        else:
            sth = float(slist[15])
                    
        if slist[16] == 'NA':
            st15 = -1
        else:
            st15 = float(slist[16])
                    
        if slist[17] == 'NA':
            st30 = -1
        else:
            st30 = float(slist[17])
                    
        if slist[18] == 'NA':
            st45 = -1
        else:
            st45 = float(slist[18])
                    
        if slist[19] == 'NA':
            st60 = -1
        else:
            st60 = float(slist[19])
                    
        if slist[20] == 'NA':
            st90 = -1
        else:
            st90 = float(slist[20])
                    
        if slist[22] == 'NA':
            minr = -1
        else:
            minr = float(slist[22])
        
        tup = (x,y,popu, sau16, sa1664, sw, sc, scp, mhh, suem, 
               mhhv, stc, stp, sth, st15, st30, st45, st60, st90, minr)
        
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I.keys():
            dict_S_I[s_id] = list()
        # add data to this s_id
        dict_S_I[s_id].append(tup)

# read in data of businesses near each grocery store
with open(dataroot+'grid_S_random_I.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        i_id = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])

        if slist[4] == 'NA':
            popu = -1
        else:
            popu = float(slist[4])

        if slist[5] == 'NA':
            sau16 = -1
        else:
            sau16 = float(slist[5])

        if slist[6] == 'NA':
            sa1664 = -1
        else:
            sa1664 = float(slist[6])
                    
        if slist[7] == 'NA':
            sw = -1
        else:
            sw = float(slist[7])
                    
        if slist[8] == 'NA':
            sc = -1
        else:
            sc = float(slist[8])
                    
        if slist[9] == 'NA':
            scp = -1
        else:
            scp = float(slist[9])
                    
        if slist[10] == 'NA':
            mhh = -1
        else:
            mhh = float(slist[10])
                    
        if slist[11] == 'NA':
            suem = -1
        else:
            suem = float(slist[11])
                    
        if slist[12] == 'NA':
            mhhv = -1
        else:
            mhhv = float(slist[12])
                    
        if slist[13] == 'NA':
            stc = -1
        else:
            stc = float(slist[13])
                    
        if slist[14] == 'NA':
            stp = -1
        else:
            stp = float(slist[14])
                    
        if slist[15] == 'NA':
            sth = -1
        else:
            sth = float(slist[15])
                    
        if slist[16] == 'NA':
            st15 = -1
        else:
            st15 = float(slist[16])
                    
        if slist[17] == 'NA':
            st30 = -1
        else:
            st30 = float(slist[17])
                    
        if slist[18] == 'NA':
            st45 = -1
        else:
            st45 = float(slist[18])
                    
        if slist[19] == 'NA':
            st60 = -1
        else:
            st60 = float(slist[19])
                    
        if slist[20] == 'NA':
            st90 = -1
        else:
            st90 = float(slist[20])
                    
        #if slist[21] == 'NA':
        #    naics = -1
        #else:
        #    naics = int(float(slist[21]))
                    
        if slist[22] == 'NA':
            minr = -1
        else:
            minr = float(slist[22])
        
        tup = (x,y,popu, sau16, sa1664, sw, sc, scp, mhh, suem, 
               mhhv, stc, stp, sth, st15, st30, st45, st60, st90, minr)

        # create entry if first time we encounter s_id
        if not s_id in dict_S_I.keys():
            dict_S_I[s_id] = list()
        # add data to this s_id
        dict_S_I[s_id].append(tup)

dict_S_I_cate1 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate1.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate1.keys():
            dict_S_I_cate1[s_id] = list()
        # add data to this s_id
        dict_S_I_cate1[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate1.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate1.keys():
            dict_S_I_cate1[s_id] = list()
        # add data to this s_id
        dict_S_I_cate1[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate2 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate2.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate2.keys():
            dict_S_I_cate2[s_id] = list()
        # add data to this s_id
        dict_S_I_cate2[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate2.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate2.keys():
            dict_S_I_cate2[s_id] = list()
        # add data to this s_id
        dict_S_I_cate2[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate3 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate3.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate3.keys():
            dict_S_I_cate3[s_id] = list()
        # add data to this s_id
        dict_S_I_cate3[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate3.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate3.keys():
            dict_S_I_cate3[s_id] = list()
        # add data to this s_id
        dict_S_I_cate3[s_id].append(tup)

#dict_S_S = dict()
dict_S_I_cate4 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate4.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate4.keys():
            dict_S_I_cate4[s_id] = list()
        # add data to this s_id
        dict_S_I_cate4[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate4.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate4.keys():
            dict_S_I_cate4[s_id] = list()
        # add data to this s_id
        dict_S_I_cate4[s_id].append(tup)

dict_S_I_cate5 = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_cate5.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate5.keys():
            dict_S_I_cate5[s_id] = list()
        # add data to this s_id
        dict_S_I_cate5[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_cate5.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_cate5.keys():
            dict_S_I_cate5[s_id] = list()
        # add data to this s_id
        dict_S_I_cate5[s_id].append(tup)

dict_S_I_comp = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_I_comp.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_comp.keys():
            dict_S_I_comp[s_id] = list()
        # add data to this s_id
        dict_S_I_comp[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_I_comp.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_cate1 = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_I_comp.keys():
            dict_S_I_comp[s_id] = list()
        # add data to this s_id
        dict_S_I_comp[s_id].append(tup)

dict_S_I_mat = dict()
for key, value in dict_S_I.items():
   
    # create a numpy array
    mat = np.empty((len(value), 20)) # 20 = 2 + 18
    for i in range(len(value)):
        #print(i)
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
        mat[i,2] = value[i][2]
        mat[i,3] = value[i][3]
        mat[i,4] = value[i][4]
        #print(value[i][4])
        mat[i,5] = value[i][5]
        mat[i,6] = value[i][6]
        mat[i,7] = value[i][7]
        mat[i,8] = value[i][8]
        mat[i,9] = value[i][9]
        mat[i,10] = value[i][10]
        mat[i,11] = value[i][11]
        mat[i,12] = value[i][12]
        mat[i,13] = value[i][13]
        mat[i,14] = value[i][14]
        mat[i,15] = value[i][15]
        mat[i,16] = value[i][16]
        mat[i,17] = value[i][17]
        mat[i,18] = value[i][18]
        mat[i,19] = value[i][19]
        
    dict_S_I_mat[key] = mat
    
dict_S_S = dict()
# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_S.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_oth = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_S.keys():
            dict_S_S[s_id] = list()
        # add data to this s_id
        dict_S_S[s_id].append(tup)

# read in data of grocery stores near each grocery store
with open(dataroot+'grid_S_random_S.csv','r') as f:
    for line in f:
        # skip header
        if line.startswith('S_id'):
            continue
        # extract data
        slist = line.strip().split(',')
        s_id = int(float(slist[0]))
        s_id_oth = int(float(slist[1]))
        x = float(slist[2])
        y = float(slist[3])
        tup = (x,y)
        # create entry if first time we encounter s_id
        if not s_id in dict_S_S.keys():
            dict_S_S[s_id] = list()
        # add data to this s_id
        dict_S_S[s_id].append(tup)

dict_S_S_mat = dict()
for key, value in dict_S_S.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_S_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_S.keys()):
    mat = np.empty((0,2))
    dict_S_S_mat[key] = mat
    
dict_S_I_cate1_mat = dict()
for key, value in dict_S_I_cate1.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate1_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate1.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate1_mat[key] = mat

dict_S_I_cate2_mat = dict()
for key, value in dict_S_I_cate2.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate2_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate2.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate2_mat[key] = mat

dict_S_I_cate3_mat = dict()
for key, value in dict_S_I_cate3.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate3_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate3.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate3_mat[key] = mat

dict_S_I_cate4_mat = dict()
for key, value in dict_S_I_cate4.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate4_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate4.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate4_mat[key] = mat

dict_S_I_cate5_mat = dict()
for key, value in dict_S_I_cate5.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_cate5_mat[key] = mat
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_cate5.keys()):
    mat = np.empty((0,2))
    dict_S_I_cate5_mat[key] = mat

dict_S_I_comp_mat = dict()
for key, value in dict_S_I_comp.items():
    # create a numpy array
    mat = np.empty((len(value), 2))
    for i in range(len(value)):
        mat[i,0] = value[i][0]
        mat[i,1] = value[i][1]
    dict_S_I_comp_mat[key] = mat
    
# regions without grocery stores:
for key in set(dict_S_I.keys()) - set(dict_S_I_comp.keys()):
    mat = np.empty((0,2))
    dict_S_I_comp_mat[key] = mat


In [ ]:
S_id_all = list(dict_S_I.keys())
# print(S_id_all)
S_id_real = [el for el in S_id_all if el<labelThresh]
# print(S_id_real)
S_id_random = [el for el in S_id_all if el>=labelThresh]
# print(S_id_random)

## Transform Data to grid (functions)

In [15]:
@numba.jit(nopython=True)
def cnt_in_cell_mat(x,y):
    out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    for i in range(len(x)):
        if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
            out[y[i],x[i]] += 1
    return out

def cnt_in_cell_mat_value(x,y,value):
    out = np.zeros((2*size_padding+size_potential,2*size_padding+size_potential))
    for i in range(len(x)):
        if min(x[i],y[i])>= 0 and max(x[i],y[i])<2*size_padding+size_potential:
            out[y[i],x[i]] += value[i]
    return out


@numba.jit(nopython=True)
def data_shift_rotate(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1):
    # rotate by theta
    theta = theta * pi / 180
    if not theta == 0:
        x = cos(theta) * mat[:,0] - sin(theta) * mat[:,1]
        y = sin(theta) * mat[:,0] + cos(theta) * mat[:,1]

    else:
        x = mat[:,0]
        y = mat[:,1]
    # mirror the region
    if mirror_var == 1 or mirror_var == -1:
        x = mirror_var * x
    # shift by shift_x, shift_y
    x = x+shift_x
    y = y+shift_y

    return x, y  

def data_to_grid(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1):

    x, y = data_shift_rotate(mat,shift_x,shift_y,theta,mirror_var)

    # fit into cells
    x = np.around(x/cell_width + size_padding).astype(int)
    y = np.around(y/cell_width + size_padding).astype(int)

    return cnt_in_cell_mat(x,y)

def data_to_grid_value(mat,shift_x=0,shift_y=0,theta=0,mirror_var=1,valueNo=2):

    x, y = data_shift_rotate(mat,shift_x,shift_y,theta,mirror_var)
    #print(x,y)
    value = mat[:,valueNo]
    #print(value)

    # fit into cells
    x = np.around(x/cell_width + size_padding).astype(int)
    y = np.around(y/cell_width + size_padding).astype(int)
    #print(x,y)

    return cnt_in_cell_mat_value(x,y,value)

def create_matLabel(y,x, gratitude = 0.15):

    if gratitude==0:
        out = np.zeros([size_potential,size_potential])
        out[y,x] = 1

        for yi in range(size_potential):
            for xi in range(size_potential):
                out[yi,xi] = 1 * mt.exp(- 1.0*((yi-y)*(yi-y)+(xi-x)*(xi-x)))
        out = np.append(out,0)
    elif gratitude==1:
        out = np.zeros(100)
        out = np.append(out,1)

    return out.flatten()

# create tensor of the proper size (1 channel currently)
grid = torch.zeros(BATCH_SIZE,nc,2*size_padding+size_potential, 2*size_padding+size_potential) #, dtype=torch.double)
labels = torch.empty(BATCH_SIZE, dtype=torch.int64)
mat_labels = torch.empty(BATCH_SIZE, size_potential*size_potential+1, dtype=torch.float)

#def create_batch(grid=grid,labels=labels,sample_ids_real=S_id_real, real_ids_weight = None, sample_ids_random=S_id_random, return_transf=False):
def create_batch(grid=grid,mat_labels=mat_labels, pos_label=pos_label,
                 logit_label=logit_label, sample_ids_real=S_id_real, real_ids_weight = None, 
                 sample_ids_random=S_id_random, return_transf=False, trainbool = 0):

    grid = grid*0

    mat_labels = mat_labels*0
    
    if return_transf:
        transf = np.zeros(shape=(BATCH_SIZE,5))

    for b in range(BATCH_SIZE):

        if b < BATCH_SIZE_real + BATCH_SIZE_fill:
            s_id = np.random.choice(sample_ids_real, p = real_ids_weight, replace=True)
        else:
            s_id = np.random.choice(sample_ids_random)
        #print(s_id)
        # get the businesses near s_id
        mat_S = dict_S_S_mat[s_id]
        mat_I = dict_S_I_mat[s_id]
        #mat_I_restaurant = dict_S_I_restaurant_mat[s_id]  
        mat_I_cate1 = dict_S_I_cate1_mat[s_id]
        mat_I_cate2 = dict_S_I_cate2_mat[s_id]
        mat_I_cate3 = dict_S_I_cate3_mat[s_id]
        mat_I_cate4 = dict_S_I_cate4_mat[s_id]
        mat_I_cate5 = dict_S_I_cate5_mat[s_id]
        mat_I_comp = dict_S_I_comp_mat[s_id]

        if trainbool==0:
            # randomly pick rotation of this region
            theta = np.random.rand()*360

            # randomly mirror? 
            mirror_var = (np.random.rand() > 0.5)*2 - 1
                #print(mirror_var)
            # randomly pick where real store is going to be
            shift_x = np.random.rand()*cell_width*size_potential - cell_width/2
            shift_y = np.random.rand()*cell_width*size_potential - cell_width/2
            
        elif trainbool==1:
            # randomly pick rotation of this region
            theta = 0

            # randomly mirror? 
            mirror_var = 1
                #print(mirror_var)
            # randomly pick where real store is going to be
            shift_x = np.random.rand()*cell_width*size_potential - cell_width/2
            shift_y = np.random.rand()*cell_width*size_potential - cell_width/2
            

        # print(shift_x,shift_y,theta,mirror_var)
        if return_transf:
            transf[b,0] = s_id
            transf[b,1] = shift_x
            transf[b,2] = shift_y
            transf[b,3] = theta
            transf[b,4] = mirror_var

        # fill tensor
        grid[b,0,:,:] = torch.from_numpy(data_to_grid(mat_S,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
        if nc >= 2:
            grid[b,1,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=2))
            grid[b,2,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=3))
            grid[b,3,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=4))
            grid[b,4,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=5))
            grid[b,5,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=6))
            grid[b,6,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=7))
            grid[b,7,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=8))
            grid[b,8,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=9))
            grid[b,9,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=10))
            grid[b,10,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=11))
            grid[b,11,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=12))
            grid[b,12,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=13))
            grid[b,13,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=14))
            grid[b,14,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=15))
            grid[b,15,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=16))
            grid[b,16,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=17))
            grid[b,17,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=18))
            grid[b,18,:,:] = torch.from_numpy(data_to_grid_value(mat_I,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var,valueNo=19))

        if nc >= 20:
            grid[b,19,:,:] = torch.from_numpy(data_to_grid(mat_I_cate1,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,20,:,:] = torch.from_numpy(data_to_grid(mat_I_cate2,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,21,:,:] = torch.from_numpy(data_to_grid(mat_I_cate3,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,22,:,:] = torch.from_numpy(data_to_grid(mat_I_cate4,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))
            grid[b,23,:,:] = torch.from_numpy(data_to_grid(mat_I_cate5,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))

        if nc >= 25:
            grid[b,24,:,:] = torch.from_numpy(data_to_grid(mat_I_comp,shift_x=shift_x,shift_y=shift_y,theta=theta,mirror_var=mirror_var))


        # include this grocery store in the covariates? (whether to gauge the point out)

        if b <= BATCH_SIZE_real:
            treat_x = int(round(shift_x/cell_width) + size_padding)
            treat_y = int(round(shift_y/cell_width) + size_padding)
            grid[b,0,treat_y,treat_x] -= 1
        

        # location of missing grocery store:
        if b <= BATCH_SIZE_real:
            mat_labels[b,:] = torch.from_numpy(create_matLabel(int(round(shift_y/cell_width)), int(round(shift_x/cell_width)),0))

        # random region without missing grocery store or grocery store is filled in:
        else:
            mat_labels[b,:] = torch.from_numpy(create_matLabel(int(round(shift_y/cell_width)), int(round(shift_x/cell_width)), 1))
            
    if not return_transf:
        return grid, mat_labels #logit_label, pos_label
    else:
        return grid, mat_labels, transf

## Define Neural Net

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        main = nn.Sequential(
            nn.InstanceNorm2d(num_features=nc, affine=True),
            nn.Conv2d(in_channels=nc,
                out_channels=2*nc,
                kernel_size=5, #9,
                padding=2, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                bias=True),
            nn.InstanceNorm2d(num_features=2*nc, affine=True),
            nn.LeakyReLU(),
           
            nn.Conv2d(in_channels=2*nc,
                out_channels=4*nc,
                kernel_size=21, # 21 #9,
                padding=20,#20  #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                dilation=2,
                bias=True),
            nn.InstanceNorm2d(num_features=4*nc, affine=True),
            nn.LeakyReLU(),
            
            nn.Conv2d(in_channels=4*nc,
                out_channels=4*nc,
                kernel_size=5, #9,
                padding=2, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                bias=True),
            nn.InstanceNorm2d(num_features=4*nc, affine=True),
            nn.LeakyReLU(),
            
            nn.Conv2d(in_channels=4*nc,
                out_channels=1, #1
                kernel_size=21, #9,
                padding=20, #4, #(9-1)/2,
                padding_mode='replicate', # 'zeros', 'reflect' or 'replicate' could work
                dilation=2,
                bias=True),

            nn.InstanceNorm2d(num_features=1, affine=True), 
            nn.Flatten(), 
            nn.Linear(1*pow(2*size_padding+size_potential,2), pow(size_potential,2)+1),  #1
        )

        self.main = main

    def forward(self, x):
        output = self.main(inputs)

        return output

## Initialize optimizer

In [18]:
def intitialize_optimizer(net):
    #return optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    #return optim.AdamW(net.parameters(), lr=0.001)
    return optim.Adam(net.parameters(), lr=0.0005)

## Code for saving and loading the model

In [19]:
def save_model(filename=None):
    if not filename:
        date = (datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d--%H-%M')
        filename = 'classifier-checkpoint-' + date + '.tar'
    path_save = dataroot + filename
    # save the model
    torch.save({
                # 'epoch': epoch,
                'net_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, path_save)
    print('file: ' + path_save)

def load_model(filename,net=None,optimizer=None):
    path_load = dataroot + filename
    if not net:
        net = Net()
    # if using GPU
    if use_cuda and torch.cuda.is_available():
        net.cuda()
    if not optimizer:
        optimizer = intitialize_optimizer(net)
    checkpoint = torch.load(path_load)
    net.load_state_dict(checkpoint['net_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return net, optimizer

## Set random seed

In [20]:
# Set random seed for reproducibility
manualSeed = 24601
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  24601


## Pick training / evaluation sample

In [21]:
# locations for training and evaluation

num_distinct_train_real = int(len(S_id_real) * frac_train_real)
num_distinct_train_random = int(len(S_id_random) * frac_train_random)

#TODO: implement such that eval regions are non-overlapping with train regions
SidRealWeight = []
for i in S_id_real:
    if i<=(fineSampleNumber-1):
        SidRealWeight.append(weight_fine)
    else:
        SidRealWeight.append(weight_plain)
        
SidRealWeight = np.array(SidRealWeight)
SidRealWeight = SidRealWeight/np.sum(SidRealWeight)

sample_train_real = list(np.random.choice(a=S_id_real, size=num_distinct_train_real, replace=True, p = SidRealWeight))
sample_train_real.sort()


sample_train_random = list(np.random.choice(a=S_id_random,size=num_distinct_train_random,replace=False))
sample_train_random.sort()

if num_distinct_train_real < len(S_id_real):
    sample_eval_real = list(set(S_id_real) - set(sample_train_real))
else:
    sample_eval_real = S_id_real
sample_eval_real.sort()

final_eval_real = [item for item in S_id_real if item<= (fineSampleNumber-1)]

# weights
train_id_weight = []
for i in sample_train_real:
    if i<=(fineSampleNumber-1):
        train_id_weight.append(weight_fine)
    else:
        train_id_weight.append(weight_plain)
        
train_id_weight = np.array(train_id_weight)
train_id_weight = train_id_weight/np.sum(train_id_weight)

eval_id_weight = []
for i in sample_eval_real:
    if i<=(fineSampleNumber-1):
        eval_id_weight.append(eval_weight_fine)
    else:
        eval_id_weight.append(eval_weight_plain)
        
eval_id_weight = np.array(eval_id_weight)
eval_id_weight = eval_id_weight/np.sum(eval_id_weight)
        
finalEval_id_weight = []
for i in final_eval_real:
    if i<=(fineSampleNumber-1):
        finalEval_id_weight.append(eval_weight_fine)
    else:
        finalEval_id_weight.append(eval_weight_plain)
        
finalEval_id_weight = np.array(finalEval_id_weight)
finalEval_id_weight = finalEval_id_weight/np.sum(finalEval_id_weight)
        

if num_distinct_train_random < len(S_id_random):
    sample_eval_random = list(set(S_id_random) - set(sample_train_random))
else:
    sample_eval_random = S_id_random
sample_eval_random.sort()

## Initialize neural net

In [24]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
        #print('h')
    elif classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


if use_saved_model:
    net, optimizer = load_model(saved_model_filename)
else:
    net = Net()
    net.apply(weights_init)
    if use_cuda and torch.cuda.is_available():
        net.cuda()
    optimizer = intitialize_optimizer(net)

criterion = nn.CrossEntropyLoss()
    
if use_cuda and torch.cuda.is_available():
    net.cuda()

## Training

In [26]:
print('Starting Training')
print((datetime.utcnow() + timedelta(hours=-4)).strftime('%Y-%m-%d %H:%M:%S'))

# loop over the dataset multiple times
print('epoch:',EPOCHS, 'iteration:', ITERS)
for epoch in range(EPOCHS):

    # initialize fit statistics

    running_loss = 0.0
    correct = 0
    correct_real = 0
    non_zero_real = 0
    correct_fill = 0
    correct_random = 0
    total = 0
    total_real = 0
    total_fill = 0
    total_random = 0
    cnt_real_about = 0

    for i in range(ITERS):
        #print(epoch,i)
        # get the inputs; data is a list of [inputs, labels]
        data = create_batch(sample_ids_real=sample_train_real, real_ids_weight = train_id_weight,
                            sample_ids_random=sample_train_random, trainbool=0)
        inputs, labels = data

        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # determine accuracy of taking "prediction"
        with torch.no_grad():
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)

            for k in range(len(predicted[0:BATCH_SIZE_real])):
                judge = predicted[k]-labels[k]
                if judge==1 or judge==-1 or judge==10 or judge==-10 or judge==11 or judge==-11 or judge==9 or judge==-9 or judge==0:
                    cnt_real_about+=1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
            non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
            correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
            correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
            total_real += BATCH_SIZE_real
            total_fill += BATCH_SIZE_fill
            total_random += BATCH_SIZE_random

        # print statistics
        running_loss += loss.item()
        if i % min(10000,ITERS/10) == min(10000,ITERS/10)-1:    # print every min(1000,ITERS/10) mini-batches
            print('[%d / %d, %5d / %5d] loss: %.3f, accuracy: %.1f%%, real: %.1f%%, real proxy: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
                  (epoch + 1, EPOCHS, i + 1, ITERS, running_loss / min(10000,ITERS/10),
                   100 * correct / total,
                   100 * correct_real / max(total_real,1),
                   100 * cnt_real_about / max(total_real,1),
                   100 * non_zero_real / max(total_real,1),
                   100 * correct_fill / max(total_fill,1),
                   100 * correct_random / max(total_random,1)))
            running_loss = 0.0
            correct = 0
            correct_real = 0
            non_zero_real = 0
            correct_fill = 0
            correct_random = 0
            total = 0
            total_real = 0
            total_fill = 0
            total_random = 0
            cnt_real_about = 0
            
        # evaluation sample:
        if frac_train_real < 1 or frac_train_random < 1:
            if i % min(10000,ITERS/2) == min(10000,ITERS/2)-1:    # print every min(1000,ITERS/10) mini-batches

                eval_correct = 0
                eval_correct_real = 0
                eval_non_zero_real = 0
                eval_correct_fill = 0
                eval_correct_random = 0
                eval_total = 0
                eval_total_real = 0
                eval_total_fill = 0
                eval_total_random = 0
                eval_cnt_real_about = 0

                with torch.no_grad():
                    for i in range(100):
                        inputs, labels = create_batch(sample_ids_real=sample_eval_real, real_ids_weight = eval_id_weight,
                                                    sample_ids_random=sample_eval_random, trainbool=1)

                        if use_cuda and torch.cuda.is_available():
                            inputs = inputs.cuda()
                            labels = labels.cuda()

                        outputs = net(inputs)
                        _, predicted = torch.max(outputs.data, 1)
                        _, labels = torch.max(labels, 1)
                        
                        for k in range(len(predicted[0:BATCH_SIZE_real])):
                            judge = predicted[k]-labels[k]
                            if judge==1 or judge==-1 or judge==10 or judge==-10 or judge==11 or judge==-11 or judge==9 or judge==-9 or judge==0:
                                eval_cnt_real_about+=1

                        eval_total += labels.size(0)
                        eval_correct += (predicted == labels).sum().item()
                        eval_correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
                        eval_non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
                        eval_correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
                        eval_correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
                        eval_total_real += BATCH_SIZE_real
                        eval_total_fill += BATCH_SIZE_fill
                        eval_total_random += BATCH_SIZE_random
                        
                print('Accuracy on hold-out: %.1f%%, real: %.1f%%, real proxy: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
                    (100 * eval_correct / max(eval_total,1),
                    100 * eval_correct_real / max(eval_total_real,1),
                    100 * eval_cnt_real_about / max(eval_total_real,1),
                    100 * eval_non_zero_real / max(eval_total_real,1),
                    100 * eval_correct_fill / max(eval_total_fill,1),
                    100 * eval_correct_random / max(eval_total_random,1)))


    print('Finished Epoch ' + str(epoch+1) + ' of ' + str(EPOCHS) + '. Saving model and optimizer checkpoint.')
    save_model()
    print((datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d %H:%M:%S'))

print('Finished Training')

Starting Training
2022-08-31 00:38:57
epoch: 10 iteration: 100
[1 / 10,    10 /   100] loss: 11.733, accuracy: 6.8%, real: 1.6%, real proxy: 10.7%, real non-zero: 76.2%, real filled: 28.1%, unrealized: 26.2%
[1 / 10,    20 /   100] loss: 10.471, accuracy: 11.2%, real: 8.4%, real proxy: 27.0%, real non-zero: 75.5%, real filled: 28.8%, unrealized: 10.0%
[1 / 10,    30 /   100] loss: 8.540, accuracy: 30.8%, real: 34.1%, real proxy: 61.6%, real non-zero: 85.8%, real filled: 17.5%, unrealized: 17.5%
[1 / 10,    40 /   100] loss: 7.166, accuracy: 48.8%, real: 55.6%, real proxy: 80.9%, real non-zero: 92.7%, real filled: 19.4%, unrealized: 26.2%
[1 / 10,    50 /   100] loss: 6.521, accuracy: 58.7%, real: 60.9%, real proxy: 83.6%, real non-zero: 94.2%, real filled: 51.9%, unrealized: 45.0%
[1 / 10,    60 /   100] loss: 6.087, accuracy: 69.8%, real: 65.5%, real proxy: 87.3%, real non-zero: 96.0%, real filled: 95.6%, unrealized: 68.8%
[1 / 10,    70 /   100] loss: 5.992, accuracy: 71.0%, real: 65

[6 / 10,    10 /   100] loss: 5.204, accuracy: 89.1%, real: 86.8%, real proxy: 94.4%, real non-zero: 99.4%, real filled: 98.8%, unrealized: 97.5%
[6 / 10,    20 /   100] loss: 5.157, accuracy: 88.1%, real: 85.4%, real proxy: 94.5%, real non-zero: 99.5%, real filled: 100.0%, unrealized: 96.2%
[6 / 10,    30 /   100] loss: 5.191, accuracy: 86.9%, real: 84.8%, real proxy: 94.3%, real non-zero: 99.4%, real filled: 98.8%, unrealized: 88.8%
[6 / 10,    40 /   100] loss: 5.062, accuracy: 87.8%, real: 85.3%, real proxy: 94.9%, real non-zero: 99.2%, real filled: 99.4%, unrealized: 95.0%
[6 / 10,    50 /   100] loss: 5.100, accuracy: 88.9%, real: 86.6%, real proxy: 95.0%, real non-zero: 99.4%, real filled: 99.4%, unrealized: 96.2%
[6 / 10,    60 /   100] loss: 5.169, accuracy: 88.7%, real: 86.1%, real proxy: 95.0%, real non-zero: 99.2%, real filled: 99.4%, unrealized: 97.5%
[6 / 10,    70 /   100] loss: 5.165, accuracy: 87.9%, real: 86.0%, real proxy: 94.3%, real non-zero: 99.7%, real filled: 98

## Evaluation

In [ ]:
print('Starting Evaluation')

eval_correct = 0
eval_correct_real = 0
eval_non_zero_real = 0
eval_correct_fill = 0
eval_correct_random = 0
eval_total = 0
eval_total_real = 0
eval_total_fill = 0
eval_total_random = 0

with torch.no_grad():
    for i in range(1000):
        print(i)
        inputs, labels = create_batch(sample_ids_real=sample_eval_real, real_ids_weight = finalEval_id_weight,
                                        sample_ids_random=sample_eval_random)

        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels, 1)

        eval_total += labels.size(0)
        
        eval_correct += (predicted == labels).sum().item()
        eval_correct_real += (predicted[0:BATCH_SIZE_real] == labels[0:BATCH_SIZE_real]).sum().item()
        eval_non_zero_real += (predicted[0:BATCH_SIZE_real] < pow(size_potential,2)).sum().item()
        eval_correct_fill += (predicted[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill] == labels[BATCH_SIZE_real:BATCH_SIZE_real+BATCH_SIZE_fill]).sum().item()
        eval_correct_random += (predicted[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE] == labels[BATCH_SIZE-BATCH_SIZE_random:BATCH_SIZE]).sum().item()
        
        eval_total_real += BATCH_SIZE_real
        eval_total_fill += BATCH_SIZE_fill
        eval_total_random += BATCH_SIZE_random

print('Accuracy on hold-out: %.1f%%, real: %.1f%%, real non-zero: %.1f%%, real filled: %.1f%%, unrealized: %.1f%%' %
        (100 * eval_correct / max(eval_total,1),
        100 * eval_correct_real / max(eval_total_real,1),
        100 * eval_non_zero_real / max(eval_total_real,1),
        100 * eval_correct_fill / max(eval_total_fill,1),
        100 * eval_correct_random / max(eval_total_random,1)))

print((datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d %H:%M:%S'))

## Functions to transform output into list for saving to file

In [27]:
@numba.jit
def data_reverse_shift_rotate(xy,shift_x=0,shift_y=0,theta=0,mirror_var=1):
    # reverse shift
    xy[:,0] -= shift_x
    xy[:,1] -= shift_y

    # reverse mirroring
    if mirror_var == 1 or mirror_var == -1:
        xy[:,0] = mirror_var * xy[:,0]
    
    # reverse rotation by theta
    theta = theta * pi / 180
    if not theta == 0:
        rot = np.array([[cos(theta),sin(theta)],[-sin(theta),cos(theta)]])
        xy = xy@np.linalg.inv(rot)
    return xy


def add_to_list(xy,o,r):
    for i in range(len(xy)):
        if i == len(xy) - 1:
            tup = (str(int(round(xy[i,0]))), str(r), 'NA', 'NA', str(o[i]))
        else:
            tup = (str(int(round(xy[i,0]))), str(r), str(int(round(xy[i,1]))), str(int(round(xy[i,2]))), str(o[i]))
        list_out.append(tup)


def outputs_to_loc(outputs,transf):
    o = outputs.cpu().numpy()
    g = np.linspace(start=cell_width/2,
                    stop=cell_width/2 + cell_width*size_potential,
                    num=size_potential, endpoint=False)
    
    for b in range(BATCH_SIZE):
        # grid cell midpoints
        xy = np.zeros(shape=(pow(size_potential,2)+1,3))
        # set s_id
        xy[:,0] = int(transf[b,0])
        # set relative location
        xy[0:pow(size_potential,2),1] = np.tile(g,size_potential)
        xy[0:pow(size_potential,2),2] = np.repeat(g,size_potential)
        xy[0:pow(size_potential,2),1:3] = data_reverse_shift_rotate(xy[0:pow(size_potential,2),1:3],
                                                                    shift_x=transf[b,1],
                                                                    shift_y=transf[b,2],
                                                                    theta=transf[b,3],
                                                                    mirror_var=transf[b,4])
        add_to_list(xy,o[b,:],b < BATCH_SIZE_real)


## Generate counterfactual locations

In [ ]:
list_out = list()

num_batches_predict = 600 # could be more to offer more choices

# change ratio of what regions to simulate since we don't care much about the real ones
BATCH_SIZE_real = 2  # regions with missing grocery store per batch #2
BATCH_SIZE_fill = 2  # regions with real location filled in (-> no missing) per batch #2
BATCH_SIZE_random = 28  # random regions (-> no missing) per batch


finalEval_id_weight = []
for i in final_eval_real:
    if i<=int(fineSampleNumber/2-1):
        finalEval_id_weight.append(1)
    else:
        finalEval_id_weight.append(1)

        
finalEval_id_weight = np.array(finalEval_id_weight)
finalEval_id_weight = finalEval_id_weight/np.sum(finalEval_id_weight)        
        

with torch.no_grad():
    for i in range(num_batches_predict):
        print(i)
        # get the inputs; data is a list of [inputs, labels]
        data = create_batch(sample_ids_real=final_eval_real,real_ids_weight = finalEval_id_weight, return_transf=True)
        inputs, labels, transf = data
        #_, labels = torch.max(labels, 1)
        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        outputs_to_loc(outputs,transf)

# print(list_out)

In [29]:
print(len(list_out))

7272000


## Save the resulting file


In [30]:
date = (datetime.utcnow() + timedelta(hours=-7)).strftime('%Y-%m-%d--%H-%M')
filename_out = 'predicted_activation-' + date + '.csv'
with open(dataroot+filename_out,'w') as f:
    f.write('S_id,real_missing,x,y,activation\n')
    for e in list_out:
        f.write(','.join(e) + '\n')